In [1]:
import sys, os, io, json
import subprocess as sp
import re
import cv2
cmd_timeout=10*60*60
FFMPEG_BINARY="C:/ffmpeg/bin/ffmpeg"

def get_video_info(file_path: str):
    """Get video information using ffmpeg."""
    print("Get video information using ffmpeg [START]")
    try:
        command = [
            FFMPEG_BINARY,
            "-i", file_path,
            "-af", "ebur128=peak=true",
            "-f", "null",
            "-"
        ]
        result = sp.run(command, capture_output=True, text=True, check=True)
        
        info = {}
        loudness_flag = False
        with open(file_path+".log", "w") as f:
            f.write(result.stderr)
        for line in result.stderr.splitlines():
            if "Duration" in line:
                match = re.search(r"Duration: (\d{2}):(\d{2}):(\d{2})\.(\d{2})", line)
                if match:
                    hours, minutes, seconds, milliseconds = map(int, match.groups())
                    info["duration_seconds"] = hours * 3600 + minutes * 60 + seconds + milliseconds / 100

                fps_match = re.search(r'(\d+)\s+fps', line)
                if fps_match:
                    info["fps"] = int(fps_match.group(1))

            if "bitrate" in line:
                match = re.search(r"bitrate: (\d+) kb/s", line)
                if match:
                    info["bitrate_kbps"] = int(match.group(1))
            elif "Stream" in line and "Video:" in line:
                width_height_match = re.search(r", (\d+)x(\d+)", line)
                if width_height_match:
                    info["width"], info["height"] = map(int, width_height_match.groups())
            elif "Summary:" in line:
                loudness_flag = True
            elif loudness_flag and "I:" in line:
                info["integrated_loudness"] = float(line.split('I:')[1].strip().split(' ')[0])
            elif loudness_flag and 'LRA:' in line:
                info["loudness_range"] = float(line.split('LRA:')[1].strip().split(' ')[0])
        
        print("Get video information using ffmpeg [DONE]")
        return info
    
    except sp.CalledProcessError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")
    
    return None

def bytes_to_str(st):
    if st is None:
        return None
    if isinstance(st, bytes):
        try:
            return  st.decode('utf-8').strip()
        except UnicodeDecodeError:
            return  st.decode('latin-1').strip()
    return str(st).strip()

def exec_cmd(cmd, timeout=cmd_timeout):
    """Exécution d'une commande shell dans un process séparé.

    Arguments:
        cmd {str} -- Commande shell à exécuter.

    Returns:
        entier -- Code retour de l'exécution de la commande :
                    str type - Succès: le stdout du commande
                    None - Echec: avec afiche sur le log l'erreur
    """
    print(f"Appel de la commande: {cmd}")
    stdout, stderr, process = None, None, None
    process = sp.Popen(str(cmd), shell=True, stderr=sp.PIPE, stdout=sp.PIPE)
    try:
        stdout, stderr = process.communicate(timeout=timeout)
    except sp.TimeoutExpired:
        # Dans cet exemple, nous lançons une commande externe avec `subprocess.Popen` et utilisons la méthode `communicate()` avec un timeout en secondes. 
        # Si le processus ne se termine pas avant l'expiration du délai, nous tuons le processus avec `process.kill()` et récupérons les sorties standard et d'erreur.
        process.kill()
        stdout, stderr = process.communicate()
        stderr=bytes_to_str(stderr)
        stdout=bytes_to_str(stdout)
        print(f"Le processus a expiré après {timeout} secondes" +
              "\n==> stderr: \n"+stderr+"\n==> stdout: \n"+stdout)
        return False, None, None
    except sp.CalledProcessError as e:
        print(f"Erreur lors de l'appel à la commande: {e}")
        return False, None, None
    except Exception as e:
        print(f"Unexpected lors de l'appel à la commande: {e}")
        return False, None, None

    returncode = int(process.returncode)
    stderr=bytes_to_str(stderr)
    stdout=bytes_to_str(stdout)
    if returncode != 0 and stderr is not None and stderr != '':
        print("Erreur lors de l'appel à la commande; returncode="+str(returncode) +
              "\n==> stderr: \n"+stderr+"\n==> stdout: \n"+stdout)
        return False, stdout, stderr
    # print("\n==> stderr: \n"+bytes_to_str(stderr)+"\n==> stdout: \n"+bytes_to_str(stdout))
    print(f"Succès de la commande {cmd}")
    return True, stdout, stderr

# Cas n 1

In [4]:
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"
expected_duration=20

output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out.mp4")
silence_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_silence.mp4")

info=get_video_info(input_video_path)
print(info)
duration=info['duration_seconds']
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count


Get video information using ffmpeg [START]


Get video information using ffmpeg [DONE]
{'duration_seconds': 19.96, 'bitrate_kbps': 16557, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.1, 'loudness_range': 5.3}
frame_rate: 25.0; frame_count: 499, expected_frame_count: 500


False

In [4]:
incorect_frame_count = expected_frame_count-frame_count
print("incorect_frame_count:",incorect_frame_count)

# Create a VideoWriter object to write the new video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(silence_video_path, fourcc, frame_rate, (width, height))

# Write the original frames to the new video except the last one
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

if expected_frame_count < frame_count:
    for _ in range(int(expected_frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
else:
    print("1")
    cnt=0
    # Write the original frames to the new video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        cnt+=1

    # Read the last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
    ret, last_frame = cap.read()

    # Write the duplicated last frame to the new video
    for _ in range(int(incorect_frame_count)):
        out.write(last_frame)
        cnt+=1

print("cnt:",cnt)
# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()


incorect_frame_count: 1
1
cnt: 500


In [7]:
cmd = f"""
{FFMPEG_BINARY} 
-i "{silence_video_path}"
-i "{input_video_path}" 
-c copy -map 0:v:0 -map 1:a:0 
-y "{output_video_path}" """
flag, _, _ = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande:  C:/ffmpeg/bin/ffmpeg  -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"  -c copy -map 0:v:0 -map 1:a:0  -y "C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4" 
Succès de la commande  C:/ffmpeg/bin/ffmpeg  -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"  -c copy -map 0:v:0 -map 1:a:0  -y "C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4" 


True

In [ ]:
pip install ffmpeg-python

In [6]:
import ffmpeg
import numpy as np

FFMPEG_BINARY="C:/ffmpeg/bin/ffmpeg"
FFMPEG_DIR_PATH="C:/ffmpeg/bin/"
# Environment setup
os.environ["FFMPEG_BINARY"] = FFMPEG_BINARY
os.environ["PATH"] += os.pathsep + FFMPEG_DIR_PATH

input_file=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"
output_file = os.path.join(os.path.dirname(os.path.abspath(input_file)),"file_out.mp4")
try:
    os.remove(output_file)
except: pass
# Get video information
probe = ffmpeg.probe(input_file)
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
duration = float(probe['format']['duration'])
frame_rate = eval(video_info['r_frame_rate'])
width = int(video_info['width'])
height = int(video_info['height'])

# Calculate number of frames to add
target_duration = 20.0
target_frames = int(target_duration * frame_rate)
frames_to_add = target_frames - int(duration * frame_rate)

# Create silent audio
silent_audio = ffmpeg.input('anullsrc', f='lavfi', t=target_duration)

# Process video
video = (
    ffmpeg
    .input(input_file)
    .filter('tpad', stop_mode='clone', stop_duration=target_duration-duration)
    .filter('fps', fps=frame_rate, round='up')
    .filter('scale', width, height)
)

# Combine video and audio
output = ffmpeg.output(video, silent_audio, output_file,
                       vcodec='libx264', acodec='aac', 
                       video_bitrate='16557k',
                       t=target_duration)

# Run FFmpeg command
ffmpeg.run(output, overwrite_output=True)

print(f"Video adjusted to {target_duration} seconds with {target_frames} frames.")

Video adjusted to 20.0 seconds with 500 frames.


In [4]:
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4"
output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out.mp4")
target_duration=20
duration=19.96
frame_rate=25
cmd = [
    FFMPEG_BINARY,
    '-i', input_video_path,
    '-filter_complex', f'[0:v]tpad=stop_mode=clone:stop_duration={target_duration-duration}[v];[0:a]apad=pad_dur={target_duration-duration}[a]',
    '-map', '[v]',
    '-map', '[a]',
    '-c:v', 'libx264', '-preset', 'slow', '-crf', '18',
    '-c:a', 'aac', '-b:a', '192k',
    '-r', str(frame_rate),
    '-t', str(target_duration),
    '-y', output_video_path
]
flag, _, _ = exec_cmd(" ".join(cmd).replace("\n", " "))
flag

Appel de la commande: C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4 -filter_complex [0:v]tpad=stop_mode=clone:stop_duration=0.03999999999999915[v];[0:a]apad=pad_dur=0.03999999999999915[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 5 -c:a aac -b:a 192k -r 25 -t 20 -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4
Succès de la commande C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4 -filter_complex [0:v]tpad=stop_mode=clone:stop_duration=0.03999999999999915[v];[0:a]apad=pad_dur=0.03999999999999915[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 5 -c:a aac -b:a 192k -r 25 -t 20 -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4


True

In [5]:
target_duration=20
cmd = [
    FFMPEG_BINARY,
    '-i', input_video_path,
    '-filter_complex', f'[0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration={target_duration-duration}[v];[0:a]apad=pad_dur={target_duration-duration}[a]',
    '-map', '[v]',
    '-map', '[a]',
    '-c:v', 'libx264', '-preset', 'slow', '-crf', '18',
    '-c:a', 'aac', '-b:a', '192k',
    '-r', str(frame_rate),
    '-t', str(target_duration),
    '-movflags', '+faststart',
    '-y', output_video_path
]
flag, _, _ = exec_cmd(" ".join(cmd).replace("\n", " "))
flag

Appel de la commande: C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration=0.03999999999999915[v];[0:a]apad=pad_dur=0.03999999999999915[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 25.0 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4
Succès de la commande C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_LITE_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration=0.03999999999999915[v];[0:a]apad=pad_dur=0.03999999999999915[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 25.0 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4


True

In [6]:
expected_duration=target_duration
print(get_video_info(output_video_path))
cap = cv2.VideoCapture(output_video_path)
# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
cap.release()
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 20.0, 'bitrate_kbps': 8209, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.1, 'loudness_range': 5.3}
frame_rate: 25.0; frame_count: 500, expected_frame_count: 500


True

# Cas n 2

## test 1

In [14]:
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"
output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out2.mp4")
expected_duration=20

info=get_video_info(input_video_path)
print(info)
duration=info['duration_seconds']
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count


Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 19.98, 'bitrate_kbps': 16699, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.5}
frame_rate: 23.976023976023978; frame_count: 479, expected_frame_count: 479


True

In [12]:
target_duration=20
cmd = [
    FFMPEG_BINARY,
    '-i', input_video_path,
    '-filter_complex', f'[0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration={target_duration-duration}[v];[0:a]apad=pad_dur={target_duration-duration}[a]',
    '-map', '[v]',
    '-map', '[a]',
    '-c:v', 'libx264', '-preset', 'slow', '-crf', '18',
    '-c:a', 'aac', '-b:a', '192k',
    '-r', str(frame_rate),
    '-t', str(target_duration),
    '-movflags', '+faststart',
    '-y', output_video_path
]
flag, _, _ = exec_cmd(" ".join(cmd).replace("\n", " "))
flag

Appel de la commande: C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration=0.019999999999999574[v];[0:a]apad=pad_dur=0.019999999999999574[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 23.976023976023978 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out2.mp4
Succès de la commande C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,tpad=stop_mode=clone:stop_duration=0.019999999999999574[v];[0:a]apad=pad_dur=0.019999999999999574[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 23.976023976023978 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out2.mp4


True

In [13]:
expected_duration=target_duration
print(get_video_info(output_video_path))
cap = cv2.VideoCapture(output_video_path)
# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
cap.release()
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 20.02, 'bitrate_kbps': 8064, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.4}
frame_rate: 23.976023976023978; frame_count: 480, expected_frame_count: 479


False

## Test 2

In [15]:
input_video_path=r"C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"
output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out2.mp4")
expected_duration=20

info=get_video_info(input_video_path)
print(info)
duration=info['duration_seconds']
cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count


Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 19.98, 'bitrate_kbps': 16699, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.5}
frame_rate: 23.976023976023978; frame_count: 479, expected_frame_count: 479


True

In [17]:
target_duration=20
cmd = [
    FFMPEG_BINARY,
    '-i', input_video_path,
    '-filter_complex', 
    # f'''
    # [0:v]minterpolate='mi_mode=mci:mc_mode=aobmc:vsbmc=1:fps=25',
    #     tpad=stop_mode=clone:stop_duration={20-19.98}[v];
    # [0:a]asetrate={48000*25/23.976023976023978},aresample=48000,
    #     apad=pad_dur={20-19.98}[a]
    # ''',
    f'[0:v]yadif=mode=1,fps=fps=25,tpad=stop_mode=clone:stop_duration={20-19.98}[v];'
    f'[0:a]asetrate={48000*25/23.976023976023978},aresample=48000,apad=pad_dur={20-19.98}[a]',
    '-map', '[v]',
    '-map', '[a]',
    '-c:v', 'libx264', '-preset', 'slow', '-crf', '18',
    '-c:a', 'aac', '-b:a', '192k',
    '-r', str(frame_rate),
    '-t', str(target_duration),
    '-movflags', '+faststart',
    '-y', output_video_path
]
flag, _, _ = exec_cmd(" ".join(cmd).replace("\n", " "))
flag

Appel de la commande: C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,fps=fps=25,tpad=stop_mode=clone:stop_duration=0.019999999999999574[v];[0:a]asetrate=50050.0,aresample=48000,apad=pad_dur=0.019999999999999574[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 23.976023976023978 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out2.mp4
Succès de la commande C:/ffmpeg/bin/ffmpeg -i C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4 -filter_complex [0:v]yadif=mode=1,fps=fps=25,tpad=stop_mode=clone:stop_duration=0.019999999999999574[v];[0:a]asetrate=50050.0,aresample=48000,apad=pad_dur=0.019999999999999574[a] -map [v] -map [a] -c:v libx264 -preset slow -crf 18 -c:a aac -b:a 192k -r 23.976023976023978 -t 20 -movflags +faststart -y C:\Users\mohammed\Desktop\workdir\app\assets\file_out2.mp4


True

In [18]:
expected_duration=target_duration
print(get_video_info(output_video_path))
cap = cv2.VideoCapture(output_video_path)
# Get video properties
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
cap.release()
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 20.02, 'bitrate_kbps': 7956, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.2, 'loudness_range': 2.4}
frame_rate: 23.976023976023978; frame_count: 480, expected_frame_count: 479


False

In [4]:
cap.release()
cmd=f'{FFMPEG_BINARY} -i "{input_video_path}" '
# cmd+=f'-filter:v "setpts=1.001*PTS,fps=25" ' # 20/19.98 = 1.001*PTS
# cmd+=f'-filter:v "fps=25,setpts=(19.98/20)*PTS" '
# cmd+=f'-filter_complex "[0:v]fps=25,split[v1][v2];[v1]trim=duration=19.98[v1trim];[v2]trim=start=19.98,setpts=PTS-STARTPTS[v2trim];[v1trim][v2trim]concat=n=2:v=1:a=0" '
# cmd+=f''' -filter_complex "[0:v]fps=25,select='if(gte(n,499),499,n)',loop=500:1:499[v]" -map "[v]" '''
# cmd+='-vf "fps=25,pad=width=iw:height=ih:color=black:duration=20" '
# cmd+=f''' -vf "fps=25,select='if(gte(n,499),499,n)',setpts=N/25/TB" -af "apad" '''
cmd+=f''' -vf "fps=25" -af "apad" -r 25 -g 12 '''
cmd+=f'-ss 00:00:00.000 -t 00:00:{expected_duration}.000 '
cmd+=f'-c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k '
cmd+=f'"{output_video_path}" -y'
# print(cmd)
flag, v1, v2 = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: C:/ffmpeg/bin/ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"  -vf "fps=25" -af "apad" -r 25 -g 12 -ss 00:00:00.000 -t 00:00:20.000 -c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k "C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4" -y
Succès de la commande C:/ffmpeg/bin/ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4"  -vf "fps=25" -af "apad" -r 25 -g 12 -ss 00:00:00.000 -t 00:00:20.000 -c:v libx264 -preset slow -crf 23 -c:a aac -b:a 128k "C:\Users\mohammed\Desktop\workdir\app\assets\file_out.mp4" -y


True

In [9]:
input_video_path="C:/Users/mohammed/Desktop/workdir/app/assets/file_out.mp4"
output_video_path = os.path.join(os.path.dirname(os.path.abspath(input_video_path)),"file_out_out.mp4")
silence_video_path = os.path.join(os.path.dirname(os.path.abspath(output_video_path)),"file_silence.mp4")
if os.path.exists(output_video_path):
    os.remove(output_video_path)
if os.path.exists(silence_video_path):
    os.remove(silence_video_path)
print(get_video_info(input_video_path))

cap = cv2.VideoCapture(input_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
# cap.release()
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Error: Command '['C:/ffmpeg/bin/ffmpeg', '-i', 'C:\\Users\\mohammed\\Desktop\\workdir\\app\\assets\\file_out_out.mp4', '-af', 'ebur128=peak=true', '-f', 'null', '-']' returned non-zero exit status 4294967294.
None
frame_rate: 0.0; frame_count: 0, expected_frame_count: 0


True

In [10]:
incorect_frame_count = expected_frame_count-frame_count
print("incorect_frame_count:",incorect_frame_count)

# Create a VideoWriter object to write the new video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(silence_video_path, fourcc, frame_rate, (width, height))

# Write the original frames to the new video except the last one
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

if expected_frame_count < frame_count:
    for _ in range(int(expected_frame_count)):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
else:
    print("1")
    cnt=0
    # Write the original frames to the new video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
        cnt+=1

    # Read the last frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_count - 1)
    ret, last_frame = cap.read()

    # Write the duplicated last frame to the new video
    for _ in range(int(incorect_frame_count)):
        out.write(last_frame)
        cnt+=1

print("cnt:",cnt)
# Release the VideoCapture and VideoWriter objects
cap.release()
out.release()


incorect_frame_count: 0
1
cnt: 0


In [24]:
FFMPEG_BINARY="ffmpeg"
cmd = f"""{FFMPEG_BINARY} -i "{silence_video_path}" -i "{input_video_path}" -c copy -map 0:v:0 -map 1:a:0 -y "{output_video_path}" """
# cmd = f"""{FFMPEG_BINARY} -i "{silence_video_path}" -i "{input_video_path}" -c copy -map 0:v -map 1:a -y "{output_video_path}" """

flag, _, _ = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -c copy -map 0:v:0 -map 1:a:0 -y "C:\Users\mohammed\Desktop\workdir\app\assets\1080p_2.mp4" 
Succès de la commande ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\file_silence.mp4" -i "C:\Users\mohammed\Desktop\workdir\app\assets\1080p.mp4" -c copy -map 0:v:0 -map 1:a:0 -y "C:\Users\mohammed\Desktop\workdir\app\assets\1080p_2.mp4" 


True

In [25]:
expected_duration=20
print(get_video_info(output_video_path))

import cv2
cap = cv2.VideoCapture(output_video_path)
# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rate = cap.get(cv2.CAP_PROP_FPS)
# Calculate the expected frame count
expected_frame_count = int(expected_duration * frame_rate)
cap.release()
print(f"frame_rate: {frame_rate}; frame_count: {frame_count}, expected_frame_count: {expected_frame_count}")
expected_frame_count==frame_count

Get video information using ffmpeg [START]
Get video information using ffmpeg [DONE]
{'duration_seconds': 20.0, 'bitrate_kbps': 10216, 'width': 1920, 'height': 1080, 'integrated_loudness': -23.3, 'loudness_range': 2.4}
frame_rate: 25.0; frame_count: 500, expected_frame_count: 500


True

In [39]:
cmd=f'{FFMPEG_BINARY} -i "{input_video_path}" '
for w, h in [(640,360), (854,480), (1280,720), (1920,1080)]:
    cmd+=f'''-f mp4 -s {w}x{h} -vf scale={w}:{h} -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a {'128k' if h<=480 else '192k'} -ss 00:00:00.000 -t 00:00:{expected_duration}.000 "C:/Users/mohammed/Desktop/workdir/app/assets/{h}p.mp4" -y '''
    # cmd+=f''' -f mp4 -c:v mpeg2video -pix_fmt yuv422p -profile:v 0 -s {w}x{h} -vf "scale={w}:{h}" -aspect 16:9 -r 25 -g 12 -c:a aac -b:a {'128k' if h<=480 else '192k'} "C:/Users/mohammed/Desktop/workdir/app/assets/{h}p.mp4" -y '''

flag, v1, v2 = exec_cmd(str(cmd).replace("\n", " "))
flag

Appel de la commande: ffmpeg -i "C:\Users\mohammed\Desktop\workdir\app\assets\pubid_FR_IPNT_BRIV_PROM_0001_020_F.mp4" -f mp4 -s 640x360 -vf scale=640:360 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 128k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/360p.mp4" -y -f mp4 -s 854x480 -vf scale=854:480 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 128k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/480p.mp4" -y -f mp4 -s 1280x720 -vf scale=1280:720 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 192k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/720p.mp4" -y -f mp4 -s 1920x1080 -vf scale=1920:1080 -aspect 16:9 -r 25 -g 12 -c:v libx264 -crf 23 -preset slow -c:a aac -b:a 192k -ss 00:00:00.000 -t 00:00:20.000 "C:/Users/mohammed/Desktop/workdir/app/assets/1080p.mp4" -y 
Succès de la commande ffmpeg -i "C:\Users\mohammed

True